In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
tf.constant(42) # 스칼라

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [ ]:
t = tf.constant([[1., 2., 3.,], [4., 5., 6.]]) # 행렬

In [ ]:
# tf.Tensor는 크기(shape)와 데이터 타입(dtype)을 가진다
t.shape, t.dtype

(TensorShape([2, 3]), tf.float32)

In [ ]:
# 인덱스 참조
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [ ]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [ ]:
print(t + 10) # tf.add(t, 10)과 동일
print(tf.square(t))
print(t @ tf.transpose(t)) # 행렬 곱셈, tf.matmul() 함수와 동일

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


In [ ]:
# 넘파이 배열로 텐서를 만들 수 있고 그 반대도 가능
a = np.array([2., 4., 5.])
print(tf.constant(a))
print(t.numpy())
print(tf.square(a))
print(np.square(t))

tf.Tensor([2. 4. 5.], shape=(3,), dtype=float64)
[[1. 2. 3.]
 [4. 5. 6.]]
tf.Tensor([ 4. 16. 25.], shape=(3,), dtype=float64)
[[ 1.  4.  9.]
 [16. 25. 36.]]


In [ ]:
# 텐서플로는 어떤 타입 변환도 자동으로 수행하지 않는다.
# 32 비트 실수와 64비트 실수는 더할 수 없다
tf.constant(2.) + tf.constant(40)
tf.constant(2.) + tf.constant(40., dtype=tf.float64)

In [ ]:
# 타입 변환 시 tf.cast() 함수를 사용
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

In [ ]:
# tf.Tensor는 변경이 불가능한 객체
# 변경이 가능한 객체는 tf.Variable을 사용
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [ ]:
# assign() 메서드를 사용하여 tf.Variable 객체의 값 변경
v.assign(2 * v)
v[0, 1].assign(42)
v[:, 2].assign([2., 1.])
v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[1.000e+02, 4.200e+01, 2.000e+00],
       [8.192e+03, 1.024e+04, 2.000e+02]], dtype=float32)>

In [ ]:
# 후버 손실
def huber_fn(y_true, y_pred):
  error = y_true - y_pred
  is_small_error = tf.abs(error) < 1
  squared_loss = tf.square(error) / 2
  linear_loss = tf.abs(error) / 2
  # is_small_error이 True: squared_loss, False: linear_loss
  return tf.where(is_small_error, squared_loss, linear_loss)

model.compile(loss=huber_fn, optimizer="nadam")
model.fit(X_train, y_train, [...])

In [ ]:
# 모델을 로드할 때는 함수 이름과 실제 함수를 매핑한 딕셔너리를 전달해야 한다.
model = keras.models.load_model("my_model_with_a_custom_loss.h5",
                                custom_objects={"huber_fn": huber_fn})

In [ ]:
# 후버 손실 매개변수
def create_huber(threshold=1.0):
  def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < threshold
    squared_loss = tf.square(error) / 2
    linear_loss = threshold * tf.abs(error) - threshold**2 / 2
    # is_small_error이 True: squared_loss, False: linear_loss
    return tf.where(is_small_error, squared_loss, linear_loss)
  return huber_fn

model.compile(loss=create_huber(2.0), optimizer="nadam")

model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5",
                                custom_objects={"huber_fn":create_huber(2.0)})

In [ ]:
# keras.losses.Loss 클래스 상속한 huber_loss
class HuberLoss(keras.losses.Loss):
  def __init__(self, threshold=1.0, **kwargs):
    sef.threshold = threshold
    super().__init__(**kwargs)
  def call(self, y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < self.threshold
    squared_loss = tf.squared(error) / 2
    linear_loss = self.threshold * abs(error) - self.threshold**2 / 2
    return tf.where(is_small_error, squared_loss, linear_loss)
  def get_config(self):
    base_config = super().get_config()
    return {**base_config, "threshold": self.threshold}

model.compile(loss=HuberLoss(2.0), optimizer="nadam")

# 저장할 때 임곗값도 함께 저장
# 모델을 로드할 때 클래스 이름과 클래스 자체를 매핑해주어야 한다.
model = keras.models.load_model("my_model_with_a_custom_loss_class.h5",
                                custom_objects={"HuberLoss": HuberLoss}))

In [ ]:
def my_softplus(z):
  return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype=tf.float32):
  stddev = tf.sqrt(2. / (shape[0] + shape[1])
  return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights):
  return tf.reduce_sum(tf.abs(0.01 + weights))

def my_positive_weights(weighs):
  return tf.where(weights < 0., tf.zeros_like(weights), weights)

layer = keras.layers.Dense(30, activation=my_softplus,
                           kernel_initializer=y_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_positive_weights)

# 함수가 모델과 함께 저장해야 할 하이퍼파라미터를 가지고 있으며
# keras.regularizers.Regularizer,
# keras.constraints.Constraint,
# keras.initializers.Initializer,
# keras.layers.Layer 와 같이 적절한 클래스를 상속

In [ ]:
# 손실함수를 지표(정확도)함수로도 사용가능
# 정밀도 계산시 keras.metrics.Precision() 클래스 사용
precision = keras.metrics.Precision()
precision(레이블, 예측)
precision.result() # 정밀도
precision.variables # 진짜 양성과 거짓양성 개수
precision.reset_states() # 두 변수가 0.0으로 리셋

In [ ]:
# 스트리밍 지표를 만들기 위해서는 keras.metrics.Metric 클래스를 상속해야한다.
class HuberMetric(keras.metrics.Metric):
  def __init__(self, threshold=1.0, **kwargs):
    super().__init__(**kwargs)
    self.threshold = threshold
    self.huber_fn = crate_huber(threshold)
    self.total = self.add_weight("total", initializer="zeros")
    self.count = self.add_weight("count", initializer="zeros")
  def update_state(self, y_true, y_pred,sample_weight=None):
    metric = self.huber_fn(y_true, y_pred)
    slef.total.assign_add(tf.reduce_sum(metric))
    self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
  def result(self):
    return self.total / self.count
  def get_config(self):
    base_config = super().get_config()
    return {**base_config, "threshold": self.threshold}

In [ ]:
# keras.layers.Flatten, keras.layers.ReLU와 같은 층은 가중치가 없다
# 가중치가 필요없는 정의 층을 만들기 가장 간단한 방법은
# 파이썬 함수를 만든 후 keras.layers.Lambda로 층을 감싸는 것이다
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

# 상태가 있는 층을 만들려면 keras.layers.Layer를 상속
class MyDense(keras.layers.Layer):
  def __init__(slef, units, activation=None, **kwargs):
    super().__init__(**kwargs)
    self.units = units
    self.activation = keras.activations.get(activation)
  
  def build(self, batch_input_shape):
    self.kernel = self.add_weight(
        name="kernel", shape=[batch_input_shape[01], self.units],
        initializer="glorot_normal")
    self.bias = self.add_weight(
        name="bias", shape=[self.units], initializer="zeros")
    super().build(batch_input_shape)
  
  def call(self, X):
    return slef.activation(X @ self.kernel + self.bias)

  def compute_output_shape(self, batch_input_shape):
    return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

  def get_config(self):
    base_config = super().get_config()
    return {**base_config, "units": self.units,
            "activation":keras.activations.serialize(self.activation)}}